# Spark_ML_Prediction

* CA Housing Data Set
* Spark EMR Notebok on Spark 2.4.4
* Recreate examples from Learning Spark 2nd Edition, Chapter 10 
  * https://www.oreilly.com/library/view/learning-spark-2nd/9781492050032/
  * Spark version 3.0 ... ML examples were converted to Spark 2.4.4
  * Use dat

In [1]:
import org.apache.spark.sql.types.{IntegerType, DoubleType, FloatType, StringType, StructField, StructType}
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1607971315187_0003,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.types.{IntegerType, DoubleType, FloatType, StringType, StructField, StructType}
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}


In [2]:
val path_train="s3://data-ca-housing/Train"
val path_test="s3://data-ca-housing/Test"

val customSchema = StructType(Array(
    StructField("longitude", DoubleType, true),
    StructField("latitude", DoubleType, true),
    StructField("housing_median_age", DoubleType, true),
    StructField("total_rooms", DoubleType, true),
    StructField("total_bedrooms", DoubleType, true),
    StructField("population", DoubleType, true),
    StructField("households", DoubleType, true),
    StructField("median_income", DoubleType, true),
    StructField("median_house_value", DoubleType, true),
    StructField("ocean_proximity", StringType, true )));

val trainDF = spark.read.format("com.databricks.spark.csv").
  option("delimiter", ",").schema(customSchema).load(path_train)

val testDF = spark.read.format("com.databricks.spark.csv").
  option("delimiter", ",").schema(customSchema).load(path_test)

trainDF.show(3)
testDF.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_train: String = s3://data-ca-housing/Train
path_test: String = s3://data-ca-housing/Test
customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(longitude,DoubleType,true), StructField(latitude,DoubleType,true), StructField(housing_median_age,DoubleType,true), StructField(total_rooms,DoubleType,true), StructField(total_bedrooms,DoubleType,true), StructField(population,DoubleType,true), StructField(households,DoubleType,true), StructField(median_income,DoubleType,true), StructField(median_house_value,DoubleType,true), StructField(ocean_proximity,StringType,true))
trainDF: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]
testDF: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|popula

**Linear Regression One Variable**

These features should go through a scaler ... the example from book did not do so, should be able to add it fairly simply

In [3]:
// Vector assembler takes a list of input columns and creates DF with features
import org.apache.spark.ml.feature.VectorAssembler 
val vecAssembler = new VectorAssembler (). 
    setInputCols(Array("median_income" )). 
    setOutputCol ( "features" ) 

val vecTrainDF = vecAssembler.transform(trainDF) 
vecTrainDF.select("median_income" , "features" , "median_house_value" ).show (10)

// Linear Regression Model
import org.apache.spark.ml.regression.LinearRegression 
val lr = new LinearRegression ().setFeaturesCol("features"). 
    setLabelCol ( "median_house_value") 

// Fit
val lrModel = lr.fit (vecTrainDF) 
val m = lrModel.coefficients(0) 
val b = lrModel.intercept 

println(f"""
The formula for the linear regression line is
  price = $m%1.2f x median_income + $b%1.2f""") 

// Create Pipeline
import org.apache.spark.ml.Pipeline 
val pipeline = new Pipeline().setStages(Array(vecAssembler,lr)) 
val pipelineModel = pipeline.fit(trainDF) 

// Apply Pipelne to Test Data
val predDF = pipelineModel.transform(testDF) 
predDF.select( "median_income" , "features" , "median_house_value" , "prediction" ).show (10) 




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.VectorAssembler
vecAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_e0f967e91029
vecTrainDF: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
+-------------+--------+------------------+
|median_income|features|median_house_value|
+-------------+--------+------------------+
|       2.3571|[2.3571]|          111400.0|
|       3.0806|[3.0806]|          106700.0|
|       3.3462|[3.3462]|           50800.0|
|       2.4805|[2.4805]|           73200.0|
|       2.0074|[2.0074]|           66900.0|
|       2.5795|[2.5795]|           68400.0|
|       3.5363|[3.5363]|           90100.0|
|        2.442| [2.442]|           69000.0|
|       3.0536|[3.0536]|          107000.0|
|       2.5096|[2.5096]|           72200.0|
+-------------+--------+------------------+
only showing top 10 rows

import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_f2a8

**Linear Regression:  Numerical + Categorical Features**

> FEATURES SHOULD BE "STANDARDIZED"

In [5]:
// Numerial and Categorical Features

import org.apache.spark.ml.feature.StringIndexer 
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val categoricalCols = "ocean_proximity"
val indexOutputCols = "ocean_proximity_index"
val oheOutputCols   = "ocean_proximity_ohe"
//val assemblerInputs = Array(oheOutputCols) ++ numericalCols

println("categoricalCols = " + categoricalCols)
println("indexOuputCols = " + indexOutputCols)
println("oheOutputCols = " + oheOutputCols)


// below works for array of categorical names, but not for just 1 so set manually below
//val stringIndexer = new StringIndexer(inputCols=categoricalCols, outputCols = indexOutputCols)
// 

// Spark 2.4.4 Indexer Output requires String input ... below OHE requires Array
//  indexer wants String input given this is for one column so must keep cols as strings 
//  then apply Array when needed
val indexer = new StringIndexer().
    setInputCol(categoricalCols).
    setOutputCol(indexOutputCols).
    setHandleInvalid("skip")

// OHE Encoder Estimator Spark vs 2.4.4 ... Transform
val oheEstimator= new OneHotEncoderEstimator().
    setInputCols(Array(indexOutputCols)).
    setOutputCols(Array(oheOutputCols))


// Some features will have nulls so need to do some imputation
// total bedrooms has nulls ... set imputation column to median
//  note, other cols are clean, but if Lat/Long had null may want to dropna
import org.apache.spark.ml.feature.Imputer
val imputer = new Imputer().
    setInputCols(Array("total_bedrooms")).
    setOutputCols(Array("total_bedrooms_imp"))

//create features
val numericalCols=Array("longitude","latitude","housing_median_age","total_rooms", "total_bedrooms_imp","population","households","median_income")

import org.apache.spark.ml.feature.VectorAssembler 
val assemblerInputs=numericalCols++Array(oheOutputCols)
val vecAssembler = new VectorAssembler(). 
    setInputCols(assemblerInputs). 
    setOutputCol ( "features" )


// Linear Regression Model
import org.apache.spark.ml.regression.LinearRegression 
val lr = new LinearRegression (). 
    setLabelCol ( "median_house_value" ). 
    setFeaturesCol ( "features" ) 

import org.apache.spark.ml.Pipeline 
val pipeline = new Pipeline(). 
    setStages (Array(indexer, oheEstimator, imputer, vecAssembler, lr))

// Fit pipeline
val pipelineModel = pipeline.fit(trainDF) 

// Apply Pipelne to Test Data
val predDF = pipelineModel.transform(testDF) 
predDF.select( "median_income" , "features" , "median_house_value" , "prediction" ).show (10) 


import org.apache.spark.ml.evaluation.RegressionEvaluator
val evaluator = new RegressionEvaluator().
  setMetricName("rmse").
  setLabelCol("median_house_value").
  setPredictionCol("prediction")

val rmse = evaluator.evaluate(predDF)
println(s"RMSE = $rmse")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
categoricalCols: String = ocean_proximity
indexOutputCols: String = ocean_proximity_index
oheOutputCols: String = ocean_proximity_ohe
categoricalCols = ocean_proximity
indexOuputCols = ocean_proximity_index
oheOutputCols = ocean_proximity_ohe
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_0da34fa48eef
oheEstimator: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_686d1d0ab551
import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_0f5abefa67ff
numericalCols: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms_imp, population, households, median_income)
import org.apache.spark.ml.feature.VectorAssembler
assemblerInputs: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms_imp, population, households, median_income, ocean_proximity_ohe

# Decision Tree Model

Decision trees are relatively fast to build, **highly interpretable**, and **scale-invariant** (i.e., standardizing or scaling the numeric features does not change the performance of the tree). 

For decision trees, you don’t have to worry about standardizing or scaling your input features, because this has no impact on the splits but you do have to be careful about how you prepare your categorical features.

Tree-based methods can naturally handle categorical variables. In spark.ml , you just need to pass the categorical columns to the StringIndexer , and the decision tree can take care of the rest. 

**Potential Error**

This produces the following error: java . lang . IllegalArgumentException : requirement failed: DecisionTree requires maxBins (= 32 ) to be at least as large as the number of values in each categorical feature , but categorical feature 3 has 36 values . Consider removing this and other categorical features with a large number of values , or add more training examples 

This produces the following error: java . lang . IllegalArgumentException : requirement failed: DecisionTree requires maxBins (= 32 ) to be at least as large as the number of values in each categorical feature , but categorical feature 3 has 36 values . Consider removing this and other categorical features with a large number of values , or add more training examples 

This produces the following error: java . lang . IllegalArgumentException : requirement failed: DecisionTree requires maxBins (= 32 ) to be at least as large as the number of values in each categorical feature , but categorical feature 3 has 36 values . Consider removing this and other categorical features with a large number of values , or add more training examples 

// In Scala 
dt.setMaxBins(40) 
val pipelineModel = pipeline.fit(trainDF) 



In [7]:
// Numerial and Categorical Features

import org.apache.spark.ml.feature.StringIndexer 
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val categoricalCols = "ocean_proximity"
val indexOutputCols = "ocean_proximity_index"
val oheOutputCols   = "ocean_proximity_ohe"
//val assemblerInputs = Array(oheOutputCols) ++ numericalCols

println("categoricalCols = " + categoricalCols)
println("indexOuputCols = " + indexOutputCols)
println("oheOutputCols = " + oheOutputCols)


// below works for array of categorical names, but not for just 1 so set manually below
//val stringIndexer = new StringIndexer(inputCols=categoricalCols, outputCols = indexOutputCols)
// 

// Spark 2.4.4 Indexer Output requires String input ... below OHE requires Array
//  indexer wants String input given this is for one column so must keep cols as strings 
//  then apply Array when needed
val indexer = new StringIndexer().
    setInputCol(categoricalCols).
    setOutputCol(indexOutputCols).
    setHandleInvalid("skip")


// Some features will have nulls so need to do some imputation
// total bedrooms has nulls ... set imputation column to median
//  note, other cols are clean, but if Lat/Long had null may want to dropna
import org.apache.spark.ml.feature.Imputer
val imputer = new Imputer().
    setInputCols(Array("total_bedrooms")).
    setOutputCols(Array("total_bedrooms_imp"))

//create features
val numericalCols=Array("longitude","latitude","housing_median_age","total_rooms", "total_bedrooms_imp","population","households","median_income")

import org.apache.spark.ml.feature.VectorAssembler 
val assemblerInputs=numericalCols++Array(indexOutputCols)
val vecAssembler = new VectorAssembler(). 
    setInputCols(assemblerInputs). 
    setOutputCol ( "features" )


// Decision Tree Regression Model
import org.apache.spark.ml.regression.DecisionTreeRegressor 
val dt = new DecisionTreeRegressor(). 
    setLabelCol ( "median_house_value" ). 
    //setMaxBins ( 40 ) 
    setFeaturesCol ( "features" ) 

import org.apache.spark.ml.Pipeline 
val pipeline = new Pipeline(). 
    setStages (Array(indexer, imputer, vecAssembler, dt))

// Fit pipeline
val pipelineModel = pipeline.fit(trainDF) 

// Apply Pipelne to Test Data
val predDF = pipelineModel.transform(testDF) 
predDF.select( "median_income" , "features" , "median_house_value" , "prediction" ).show (10) 


import org.apache.spark.ml.evaluation.RegressionEvaluator
val evaluator = new RegressionEvaluator().
  setMetricName("rmse").
  setLabelCol("median_house_value").
  setPredictionCol("prediction")

val rmse = evaluator.evaluate(predDF)
println(s"RMSE = $rmse")

val dtModel = pipelineModel.stages.
    last.asInstanceOf[org.apache.spark.ml.regression.DecisionTreeRegressionModel]

val featureImp = vecAssembler.getInputCols.zip(dtModel.featureImportances.toArray) 
val columns = Array ( "feature" , "Importance" ) 
val featureImpDF = spark.createDataFrame (featureImp). 
    toDF (columns : _ * ) 


import org.apache.spark.ml.evaluation.RegressionEvaluator
val evaluator = new RegressionEvaluator().
  setMetricName("rmse").
  setLabelCol("median_house_value").
  setPredictionCol("prediction")

val rmse = evaluator.evaluate(predDF)
println(s"RMSE = $rmse")

featureImpDF.orderBy($"Importance".desc ).show () 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
categoricalCols: String = ocean_proximity
indexOutputCols: String = ocean_proximity_index
oheOutputCols: String = ocean_proximity_ohe
categoricalCols = ocean_proximity
indexOuputCols = ocean_proximity_index
oheOutputCols = ocean_proximity_ohe
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_54cf0987c446
import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_6ae604ec1656
numericalCols: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms_imp, population, households, median_income)
import org.apache.spark.ml.feature.VectorAssembler
assemblerInputs: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms_imp, population, households, median_income, ocean_proximity_index)
vecAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_5ce6371fda63
impo


# Random Forest w/ HyperParameter Search

In [12]:
// Do this Later


import org.apache.spark.ml.feature.StringIndexer 
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val categoricalCols = "ocean_proximity"
val indexOutputCols = "ocean_proximity_index"
val oheOutputCols   = "ocean_proximity_ohe"
//val assemblerInputs = Array(oheOutputCols) ++ numericalCols

println("categoricalCols = " + categoricalCols)
println("indexOuputCols = " + indexOutputCols)
println("oheOutputCols = " + oheOutputCols)


// below works for array of categorical names, but not for just 1 so set manually below
//val stringIndexer = new StringIndexer(inputCols=categoricalCols, outputCols = indexOutputCols)
// 

// Spark 2.4.4 Indexer Output requires String input ... below OHE requires Array
//  indexer wants String input given this is for one column so must keep cols as strings 
//  then apply Array when needed
val indexer = new StringIndexer().
    setInputCol(categoricalCols).
    setOutputCol(indexOutputCols).
    setHandleInvalid("skip")


// Some features will have nulls so need to do some imputation
// total bedrooms has nulls ... set imputation column to median
//  note, other cols are clean, but if Lat/Long had null may want to dropna
import org.apache.spark.ml.feature.Imputer
val imputer = new Imputer().
    setInputCols(Array("total_bedrooms")).
    setOutputCols(Array("total_bedrooms_imp"))

//create features
val numericalCols=Array("longitude","latitude","housing_median_age","total_rooms", "total_bedrooms_imp","population","households","median_income")

import org.apache.spark.ml.feature.VectorAssembler 
val assemblerInputs=numericalCols++Array(indexOutputCols)
val vecAssembler = new VectorAssembler(). 
    setInputCols(assemblerInputs). 
    setOutputCol ( "features" )


// Random Forest Regressor
import org.apache.spark.ml.regression.RandomForestRegressor 
val rf = new RandomForestRegressor().
    setLabelCol("median_house_value"). 
    //setMaxBins (40). 
    setFeaturesCol ( "features" ) 

import org.apache.spark.ml.Pipeline 
val pipeline = new Pipeline(). 
    setStages (Array(indexer, imputer, vecAssembler, rf))

// Hyperparameter search Grid
import org.apache.spark.ml.tuning.ParamGridBuilder 
val paramGrid = new ParamGridBuilder(). 
    addGrid (rf.maxDepth, Array(2 , 4 , 6 )). 
    addGrid (rf.numTrees, Array(10, 100)). 
    build () 

import org.apache.spark.ml.evaluation.RegressionEvaluator
val evaluator = new RegressionEvaluator().
  setMetricName("rmse").
  setLabelCol("median_house_value").
  setPredictionCol("prediction")

// Cross Validation
import org.apache.spark.ml.tuning.CrossValidator 
val cv = new CrossValidator(). 
    setEstimator(pipeline). 
    setEvaluator (evaluator). 
    setEstimatorParamMaps(paramGrid). 
    setNumFolds (3). 
    setSeed (42) 

val cvModel = cv.fit (trainDF) 

cvModel.getEstimatorParamMaps.zip(cvModel.avgMetrics)

// So, how many models did we just rain? If you answered 18 (6 = 3 x 2 )
// Once you've identified the optimal hyperparameter configuration, how do you
// combine those three (or k) models together? While some models might be easy
// enough to average together, some are not. Therefore, Spark retrains your model
// on the entire training data set once it has identified the optimal hyperparameter
// configuration, so in the end we trained 19 models.
// if you want to retrain teh intermediate models trained, you can set colectSubModes=True
// in the CrossValidator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.OneHotEncoderEstimator
categoricalCols: String = ocean_proximity
indexOutputCols: String = ocean_proximity_index
oheOutputCols: String = ocean_proximity_ohe
categoricalCols = ocean_proximity
indexOuputCols = ocean_proximity_index
oheOutputCols = ocean_proximity_ohe
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_859dfa7b36ae
import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_0f40dc2fe5c5
numericalCols: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms_imp, population, households, median_income)
import org.apache.spark.ml.feature.VectorAssembler
assemblerInputs: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms_imp, population, households, median_income, ocean_proximity_index)
vecAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_41ef137e85c9
impo